In [17]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv() 

user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')  
database = 'fininsurance'

engine = create_engine(f'mariadb+mariadbconnector://{user}:{password}@{host}:{port}/{database}')

query = """
SELECT ic_ins.cuscode, COUNT(*) AS cnt
FROM tbl_ins_card ic_ins
WHERE ic_ins.ins_type = 'INS'
  AND ic_ins.cuscode LIKE 'FNG%'
  AND ic_ins.name NOT LIKE '%ทดสอบ%'
  AND ic_ins.name NOT LIKE '%test%'
  AND ic_ins.name NOT LIKE '%เทสระบบ%'
  AND ic_ins.name NOT LIKE '%Tes ระบบ%'
  AND ic_ins.name NOT LIKE '%ทด่ท%'
  AND ic_ins.name NOT LIKE '%ทด สอบ%'
GROUP BY ic_ins.cuscode
HAVING COUNT(*) > 1;
"""

df2 = pd.read_sql(query, engine)
df2


,cuscode,cnt
0,FNG20-0054,2
1,FNG20-5494,2
2,FNG20-6936,2
3,FNG20-8224,2
4,FNG20-8628,2
...,...,...
360,FNG25-139979,2
361,FNG25-140016,2
362,FNG25-140019,2
363,FNG25-140032,2


In [12]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

load_dotenv() 

user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')  
database = 'fininsurance'

engine = create_engine(f'mariadb+mariadbconnector://{user}:{password}@{host}:{port}/{database}')

query = """
SELECT 
    ic_ins.cuscode AS agent_id,
    ic_ins.title,
    CONCAT(ic_ins.name, ' ', ic_ins.lastname) AS agent_name,

    -- ข้อมูลจากประกันวินาศภัย
    ic_ins.card_no AS id_card_ins,
    ic_ins.type AS type_ins,
    ic_ins.revoke_type_code AS revoke_type_ins,
    STR_TO_DATE(NULLIF(ic_ins.create_date, '0000-00-00'), '%Y-%m-%d') AS card_issued_date_ins,
    STR_TO_DATE(NULLIF(ic_ins.expire_date, '0000-00-00'), '%Y-%m-%d') AS card_expiry_date_ins,

    -- ข้อมูลจากประกันชีวิต
    ic_life.card_no AS id_card_life,
    ic_life.type AS type_life,
    ic_life.revoke_type_code AS revoke_type_life,
    STR_TO_DATE(NULLIF(ic_life.create_date, '0000-00-00'), '%Y-%m-%d') AS card_issued_date_life,
    STR_TO_DATE(NULLIF(ic_life.expire_date, '0000-00-00'), '%Y-%m-%d') AS card_expiry_date_life

FROM tbl_ins_card ic_ins
LEFT JOIN tbl_ins_card ic_life
    ON ic_life.cuscode = ic_ins.cuscode AND ic_life.ins_type = 'LIFE'

WHERE ic_ins.ins_type = 'INS'
    AND ic_ins.cuscode LIKE 'FNG%'
    AND ic_ins.name NOT LIKE '%ทดสอบ%'
    AND ic_ins.name NOT LIKE '%test%'
    AND ic_ins.name NOT LIKE '%เทสระบบ%'
    AND ic_ins.name NOT LIKE '%Tes ระบบ%'
    AND ic_ins.name NOT LIKE '%ทด่ท%'
    AND ic_ins.name NOT LIKE '%ทด สอบ%';

"""

df = pd.read_sql(query, engine)
df


,agent_id,title,agent_name,id_card_ins,type_ins,revoke_type_ins,card_issued_date_ins,card_expiry_date_ins,id_card_life,type_life,revoke_type_life,card_issued_date_life,card_expiry_date_life
0,FNG1912-0045-defect,น.ส.,ไอลดา รามนต์,6204024773,B,,2019-06-19,2027-06-18,None,None,None,None,None
1,FNG20-0002-defect,นาย,ใจเพชร ชุมผาง,6204043817,B,,2019-10-03,2027-10-02,6701025521,A,,2024-05-16,2025-05-15
2,FNG20-0034-defect,นาย,กิตติพัฒน์ รัตนพงษ์,5904021317,B,A,2016-08-02,2024-08-01,None,None,None,None,None
3,FNG20-0232-defect,นาย,นิอานัน สามะอาลี,6204028792,B,,2019-07-12,2027-07-11,6701001152,A,A,2024-01-17,2025-01-16
4,FNG20-0278-defect,นาง,นาคิรินทร์ คงเกษม,6204014123,B,,2019-04-02,2027-04-01,6701038414,A,,2024-07-09,2025-07-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24475,FNG25-140032,น.ส.,สาวิตรี เฮงพระพรหม,5704007248,B,None,2014-04-03,2027-04-02,5603005991,B,None,2013-05-28,2026-05-27
24476,FNG25-140032,น.ส.,สาวิตรี เฮงพระพรหม,5603005991,B,None,2013-05-28,2026-05-27,5603005991,B,None,2013-05-28,2026-05-27
24477,FNG25-140033,น.ส.,จันทร์จิรา โพธิ์ศรี,6404009881,B,None,2021-11-26,2029-11-25,None,None,None,None,None
24478,FNG25-140033,น.ส.,จันทร์จิรา โพธิ์ศรี,6203023564,B,None,2019-09-18,2027-09-17,None,None,None,None,None


In [ ]:
24480, 24425 

In [11]:
df = df.drop_duplicates()
df

,cuscode,title,agent_name,id_card_ins,type_ins,revoke_type_ins,card_issued_date_ins,card_expiry_date_ins,id_card_life,type_life,revoke_type_life,card_issued_date_life,card_expiry_date_life
0,FNG1912-0045-defect,น.ส.,ไอลดา รามนต์,6204024773,B,,2019-06-19,2027-06-18,None,None,None,None,None
1,FNG20-0002-defect,นาย,ใจเพชร ชุมผาง,6204043817,B,,2019-10-03,2027-10-02,6701025521,A,,2024-05-16,2025-05-15
2,FNG20-0034-defect,นาย,กิตติพัฒน์ รัตนพงษ์,5904021317,B,A,2016-08-02,2024-08-01,None,None,None,None,None
3,FNG20-0232-defect,นาย,นิอานัน สามะอาลี,6204028792,B,,2019-07-12,2027-07-11,6701001152,A,A,2024-01-17,2025-01-16
4,FNG20-0278-defect,นาง,นาคิรินทร์ คงเกษม,6204014123,B,,2019-04-02,2027-04-01,6701038414,A,,2024-07-09,2025-07-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24475,FNG25-140032,น.ส.,สาวิตรี เฮงพระพรหม,5704007248,B,None,2014-04-03,2027-04-02,5603005991,B,None,2013-05-28,2026-05-27
24476,FNG25-140032,น.ส.,สาวิตรี เฮงพระพรหม,5603005991,B,None,2013-05-28,2026-05-27,5603005991,B,None,2013-05-28,2026-05-27
24477,FNG25-140033,น.ส.,จันทร์จิรา โพธิ์ศรี,6404009881,B,None,2021-11-26,2029-11-25,None,None,None,None,None
24478,FNG25-140033,น.ส.,จันทร์จิรา โพธิ์ศรี,6203023564,B,None,2019-09-18,2027-09-17,None,None,None,None,None


In [19]:
output_path = "dim_card_agent.xlsx"
df2.to_excel(output_path, index=False, engine='openpyxl')
print(f"💾 Saved to {output_path}")

💾 Saved to dim_card_agent.xlsx
